# Recommending movies: retrieval
- https://www.tensorflow.org/recommenders/examples/basic_retrieval
    
Real-world recommender systems
1. retrieval: selecting an initial set of hundreds of candidates from all possible candidates
2. ranking: the outputs of the retrieval model -> the best possible handful of recommendations

Retrieval의 두 가지 sub-model
1. 쿼리 모델: the query representation (normally a fixed-dimensionality embedding vector)
2. 후보 모델: the candidate representation (an equally-sized vector)

쿼리모델, 후보모델 결과를 곱해서 -> 쿼리-후보간 선호도 점수 측정

implicit feedback vs. explicit feedback

In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets
!pip install -q scann

You should consider upgrading via the '/Users/wonji/.pyenv/versions/3.7.7/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/wonji/.pyenv/versions/3.7.7/bin/python3.7 -m pip install --upgrade pip' command.
ERROR: Could not find a version that satisfies the requirement scann
ERROR: No matching distribution found for scann
You should consider upgrading via the '/Users/wonji/.pyenv/versions/3.7.7/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import os 
import pprint
import tempfile
from typing import Dict, Text
import numpy as np 
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs

In [3]:
# Ratings data.
ratings = tfds.load("movielens/100k-ratings", split="train")
# Features of all the available movies.
movies = tfds.load("movielens/100k-movies", split="train")

In [4]:
for x in ratings.take(1).as_numpy_iterator():
    pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}


In [6]:
for x in movies.take(1).as_numpy_iterator():
    pprint.pprint(x)

{'movie_genres': array([4]),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}


In [7]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
})
movies = movies.map(lambda x: x["movie_title"])

In [9]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)
# train 8 test 2
train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [10]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

In [11]:
unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

### query model 

In [12]:
embedding_dimension = 32

In [13]:
# user_ids -> integer -> embedding
user_model = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=unique_user_ids, mask_token=None),
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

### candidate model

In [14]:
movie_model = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

### metrics & loss

In [15]:
metrics = tfrs.metrics.FactorizedTopK(candidates=movies.batch(128).map(movie_model))
task = tfrs.tasks.Retrieval(metrics=metrics)